In [13]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations


In [ ]:
def get_difference(data, num_feat):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = df[num_feat].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_feat].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    return df1


# Read & preprocess data and save it

def read_preprocess_data():
    train = pd.read_parquet('train.parquet')
    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_feat = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    num_feat = [col for col in features if col not in cat_feat]
    print('Starting training feature engineer...')
    train_num_agg = train.groupby("customer_ID")[num_feat].agg(['mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)
    train_cat_agg = train.groupby("customer_ID")[cat_feat].agg(['count', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)
    train_labels = pd.read_parquet('train_labels.parquet')
    # Transform float64 columns to float32
    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        train_num_agg[col] = train_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        train_cat_agg[col] = train_cat_agg[col].astype(np.int32)
    # Get the difference
    train_diff = get_difference(train, num_feat)
    train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')
    del train_num_agg, train_cat_agg, train_diff
    gc.collect()
    train.to_parquet('train_lgbt1.parquet')

# Read & Preprocess Data
read_preprocess_data()
print('train_lgbt1 has been completed')

In [ ]:
def get_difference1(data, num_feat):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = df[num_feat].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_feat].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    return df1

def predproc_for_test():
    test = pd.read_parquet('test_data.parquet')
    features = test.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_feat = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68",]
    num_feat = [col for col in features if col not in cat_feat]
    print('Starting test feature engineer...')
    test_num_agg = test.groupby("customer_ID")[num_feat].agg(['mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    test_num_agg.reset_index(inplace = True)
    test_cat_agg = test.groupby("customer_ID")[cat_feat].agg(['count', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    test_cat_agg.reset_index(inplace = True)
    # Transform float64 columns to float32
    cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
       test_num_agg[col] = test_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        test_cat_agg[col] = test_cat_agg[col].astype(np.int32)
    # Get the difference
    test_diff = get_difference1(test, num_feat)
    test = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID').merge(test_diff, how = 'inner', on = 'customer_ID')
    del test_num_agg, test_cat_agg, test_diff
    gc.collect()
    # Save files to disk
    test.to_parquet('test_lgbt1.parquet')


predproc_for_test()

In [14]:
train = pd.read_parquet('train_lgbt1.parquet')

In [15]:
train.drop(axis = 1, columns=['customer_ID'], inplace = True)

In [ ]:
def read_data():
    train = pd.read_parquet('train_lgbt1.parquet')
    test  = pd.read_parquet('test_lgbt1.parquet')
    return train, test
def encoding(train, test):
    # Label encode categorical features
    cat_feat = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ]
    cat_feat = [f"{cf}_last" for cf in cat_feat]
    print('Label encoding...')
    for cat_column in cat_feat:
        encod = LabelEncoder()
        train[cat_column] = encod.fit_transform(train[cat_column])
        #test[cat_column] = encod.fit_transform(train[cat_column])
    # Round last float features to 2 decimal place
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    print('roundin...')
    for col in num_cols:
        train[col + '_round2'] = train[col].round(2)
        # test[col + '_round2'] = test[col].round(2)
    # Get the difference between last and mean
    num_cols = [col for col in train.columns if 'last' in col]
    num_cols = [col[:-5] for col in num_cols if 'round' not in col]
    print('last mean diff...')
    for col in num_cols:
        try:
            train[f'{col}_last_mean_diff'] = train[f'{col}_last'] - train[f'{col}_mean']
            #test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
        except:
            pass

In [16]:
train_labels = pd.read_parquet('train_labels.parquet')
train_labels.drop(columns=['customer_ID'], inplace=True)
y = np.ravel(train_labels)
del train_labels
gc.collect()

2026

In [ ]:
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)
# ====================================================
# LGBM amex metric
# ====================================================
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True


In [ ]:

import lightgbm as lgb

class CONFIG:
    seed = 42
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'

############
print('readin')
train, test = read_data()
print('encoding')
encoding(train, test)

############
train.drop( axis = 1, columns = ['customer_ID'], inplace = True)
print('featuring')
features = [col for col in train.columns if col not in ['customer_ID', CONFIG.target]]
d_train = lgb.Dataset(train[features], y)
params = {
    'objective': 'binary',
    'metric': CONFIG.metric,
    'boosting': CONFIG.boosting_type,
    'seed': CONFIG.seed,
    'num_leaves': 100,
    'learning_rate': 0.01,
    'feature_fraction': 0.20,
    'bagging_freq': 10,
    'bagging_fraction': 0.50,
    'n_jobs': -1,
    'lambda_l2': 2,
    'min_data_in_leaf': 40,
}
print('modelling')

model = lgb.train(params = params,
                  train_set = d_train,
                  num_boost_round = 10500,
                  early_stopping_rounds = 1500,
                  verbose_eval = 500,
                  feval = lgb_amex_metric)
del train
del y
gc.collect()

joblib.dump(model, 'model.pkl')
gc.collect()


In [ ]:
import joblib
joblib.dump(model, 'ft_model.joblib')
gc.collect()

In [17]:
test= pd.read_parquet('test_lgbt1.parquet')
#test['D_64_last']

In [18]:
def encoding_test(test):
    cat_feat = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ]
    cat_feat = [f"{cf}_last" for cf in cat_feat]
    print('Label encoding...')
    for cat_column in cat_feat:
        encod = LabelEncoder()
        test[cat_column] = encod.fit_transform(test[cat_column])
    num_cols = list(test.dtypes[(test.dtypes == 'float32') | (test.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    print('roundin...')
    for col in num_cols:
        test[col + '_round2'] = test[col].round(2)
    # Get the difference between last and mean
    num_cols = [col for col in test.columns if 'last' in col]
    num_cols = [col[:-5] for col in num_cols if 'round' not in col]
    print('last mean diff...')
    for col in num_cols:
        try:
            test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
        except:
            pass
encoding_test(test)

Label encoding...
roundin...
last mean diff...


In [19]:
model = joblib.load('model.pkl')
test.drop(axis = 1, columns = ['customer_ID'], inplace = True)


In [20]:
f = test.columns.to_list()
f

['P_2_mean',
 'P_2_std',
 'P_2_min',
 'P_2_max',
 'P_2_last',
 'D_39_mean',
 'D_39_std',
 'D_39_min',
 'D_39_max',
 'D_39_last',
 'B_1_mean',
 'B_1_std',
 'B_1_min',
 'B_1_max',
 'B_1_last',
 'B_2_mean',
 'B_2_std',
 'B_2_min',
 'B_2_max',
 'B_2_last',
 'R_1_mean',
 'R_1_std',
 'R_1_min',
 'R_1_max',
 'R_1_last',
 'S_3_mean',
 'S_3_std',
 'S_3_min',
 'S_3_max',
 'S_3_last',
 'D_41_mean',
 'D_41_std',
 'D_41_min',
 'D_41_max',
 'D_41_last',
 'B_3_mean',
 'B_3_std',
 'B_3_min',
 'B_3_max',
 'B_3_last',
 'D_42_mean',
 'D_42_std',
 'D_42_min',
 'D_42_max',
 'D_42_last',
 'D_43_mean',
 'D_43_std',
 'D_43_min',
 'D_43_max',
 'D_43_last',
 'D_44_mean',
 'D_44_std',
 'D_44_min',
 'D_44_max',
 'D_44_last',
 'B_4_mean',
 'B_4_std',
 'B_4_min',
 'B_4_max',
 'B_4_last',
 'D_45_mean',
 'D_45_std',
 'D_45_min',
 'D_45_max',
 'D_45_last',
 'B_5_mean',
 'B_5_std',
 'B_5_min',
 'B_5_max',
 'B_5_last',
 'R_2_mean',
 'R_2_std',
 'R_2_min',
 'R_2_max',
 'R_2_last',
 'D_46_mean',
 'D_46_std',
 'D_46_min',


In [21]:
sample = pd.read_csv('sample_submission.csv')
predictions = model.predict(test)
output_file = pd.DataFrame({'customer_ID': sample.customer_ID, 'prediction': predictions})
output_file.to_csv('lgbt.csv', index= False)

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns
def plotImp(model, f , num = 20, fig_size = (100, 80)):
    feature_imp = pd.DataFrame({'Value':model.feature_importance(),'Feature':f})
    plt.figure(figsize=fig_size)
    sns.set(font_scale = 5)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:num])
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances-01.png')
    plt.show()
plotImp(model, f, 20, (100,80))